In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("/kaggle/input/feathers-in-focus-model/"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/class_names.npy
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/test_images_path.csv
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/attributes.npy
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/test_images_sample.csv
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images.csv
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/attributes.txt
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/1269.jpg
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/3863.jpg
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/623.jpg
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/2193.jpg
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/3750.jpg
/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/trai

**Data preparation**

Creating a dataset that is compatible with torch.DataLoader

In [2]:
# importing packages

import numpy as np
import pandas as pd
import torch as tc

from pathlib import Path
from PIL import Image

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2

Transforming images to make them compatible for CNN

In [3]:
TRANSFORM_DEFAULT = v2.Compose([
    v2.ToImage(),
    v2.Resize((224, 224)),
    v2.ToDtype(tc.float32, scale=True), 
])

In [4]:
DATA_DIR  = Path("/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus")

In [5]:
train_df = pd.read_csv("/kaggle/input/feathers-in-focus-model//aml-2025-feathers-in-focus/train_images.csv")
print(train_df.head())

            image_path  label
0  /train_images/1.jpg      1
1  /train_images/2.jpg      1
2  /train_images/3.jpg      1
3  /train_images/4.jpg      1
4  /train_images/5.jpg      1


Defining the type and config parameters of the data

In [6]:
# idx, image, label, path

ItemType = tuple[int, tc.Tensor, int, str]

Define how to read a sample given the index

In [7]:
class ImageClassification(Dataset[ItemType]):
    def __init__(
        self,
        df: pd.DataFrame,
        search_root: Path,
        transform: v2.Transform = TRANSFORM_DEFAULT,
    ):
        self.df = df.reset_index(drop=True)
        self.search_root = search_root
        self.transform = transform
    
    def __len__(self) -> int:
        return len(self.df)
    
    def _find_image_path(self, filename: str) -> Path:
        path = self.search_root / filename.lstrip('/')
        if not path.exists():
            raise FileNotFoundError(f"Image file '{filename}' not found at {path}")
        return path
    
    def __getitem__(self, idx: int) -> ItemType:
        row = self.df.iloc[idx]
        
        filename = Path(str(row["image_path"])).name
        
        path = self._find_image_path(filename)
        
        image = Image.open(path).convert("RGB")
        image = self.transform(image)
        
        label = int(row["label"])
        
        return idx, image, label, str(path)

Check if set-up works

In [8]:
SEARCH_ROOT = Path("/kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images")

train_dataset = ImageClassification(
    df=train_df,
    search_root=SEARCH_ROOT,
    transform=TRANSFORM_DEFAULT,
)

idx0, img0, label0, path0 = train_dataset[0]
print("Pad eerste image:", path0)
print("Image shape:", img0.shape)
print("Label:", label0)

Pad eerste image: /kaggle/input/feathers-in-focus-model/aml-2025-feathers-in-focus/train_images/1.jpg
Image shape: torch.Size([3, 224, 224])
Label: 1
